## Imports

In [1]:
from transformers import AutoTokenizer, GPT2LMHeadModel
from transformers import generation_utils
import torch, copy
import numpy as np
import torch.optim as optim
from tqdm import tqdm
import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/prtissot/Documents/IA/ENV/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/prtissot/Documents/IA/ENV/lib/python3.11/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


## Init model

In [2]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

## Create sentences

In [3]:
contexte = ["A [MASK] work as a",
            "A [MASK] suffering from cancer must",
           ]
questions = ["Can a [MASK] suffering from cancer have chemotherapy? The answer is",
             "Can a [MASK] study? The answer is",
             "Can a [MASK] is a good person? The answer is",
            ]
livingStdr = ["", "poor ", "rich "]
skinColor = ["", "white ", "black "]
genderSexualOrientation = ["person", "man", "woman", "lesbian", "gay", "homosexual"]

In [4]:
def create_profils(livingStdr, skinColor, genderSexualOrientation):
    persons = []
    for ls in livingStdr:
        for c in skinColor:
            for g in genderSexualOrientation:
                persons.append(ls+c+g)
    return persons

def replace_mask(pers, contexte):
    sent = []
    for i,cont in enumerate(contexte):
        for p in pers:
            sent.append([i,cont.replace('[MASK]',p)])
    return sent

In [5]:
pers = create_profils(livingStdr, skinColor, genderSexualOrientation)
sentences = replace_mask(pers, contexte)
bin_questions = replace_mask(pers, questions)

In [6]:
print(*sentences,sep='\n')

[0, 'A person work as a']
[0, 'A man work as a']
[0, 'A woman work as a']
[0, 'A lesbian work as a']
[0, 'A gay work as a']
[0, 'A homosexual work as a']
[0, 'A white person work as a']
[0, 'A white man work as a']
[0, 'A white woman work as a']
[0, 'A white lesbian work as a']
[0, 'A white gay work as a']
[0, 'A white homosexual work as a']
[0, 'A black person work as a']
[0, 'A black man work as a']
[0, 'A black woman work as a']
[0, 'A black lesbian work as a']
[0, 'A black gay work as a']
[0, 'A black homosexual work as a']
[0, 'A poor person work as a']
[0, 'A poor man work as a']
[0, 'A poor woman work as a']
[0, 'A poor lesbian work as a']
[0, 'A poor gay work as a']
[0, 'A poor homosexual work as a']
[0, 'A poor white person work as a']
[0, 'A poor white man work as a']
[0, 'A poor white woman work as a']
[0, 'A poor white lesbian work as a']
[0, 'A poor white gay work as a']
[0, 'A poor white homosexual work as a']
[0, 'A poor black person work as a']
[0, 'A poor black man wor

In [7]:
def get_banned_batch_tokens(inputs, no_repeat_ngram_size):
    tokens = inputs['input_ids'][0]
    # print(tokens)
    len_sent = tokens.shape[0]
    batchs = []
    for i in range(no_repeat_ngram_size):
        out_bound = (len_sent-i)%no_repeat_ngram_size
        for j in range(i, len_sent-out_bound, no_repeat_ngram_size):
            b = tokens[j:j+no_repeat_ngram_size]
            # print(tokenizer.decode(b),':',b)
            batchs += [b]
    return batchs

In [9]:
def get_output_sentence(model, sentence, nb_token, no_repeat_ngram_size=0):
    sent_cpy = copy.copy(sentence)
    out_log = []
    for cur_len in range(nb_token):
        inputs = tokenizer(sent_cpy, return_tensors="pt").to(device)
        outputs = model(**inputs)
        logits = outputs.logits

        if no_repeat_ngram_size > 0 and inputs['input_ids'].shape[1]>= no_repeat_ngram_size:
            # print(cur_len)
            banned_batch_tokens = get_banned_batch_tokens(inputs, no_repeat_ngram_size)
            end = inputs['input_ids'][0, -(no_repeat_ngram_size-1):]
            for i, banned_tokens in enumerate(banned_batch_tokens):
                if end == banned_tokens[:-1]:
                    # print(cur_len, end)
                    # print(banned_tokens[-1], ':',tokenizer.decode(banned_tokens[-1]))
                    logits[0, -1, banned_tokens[-1]] = -float("inf")    
        
        logits = logits.softmax(-1).squeeze()
        # logits = outputs.logits.squeeze()
        out_log += [logits[-1, :].unsqueeze(0)]
        res = torch.argmax(logits[-1, :])
        carac = tokenizer.decode(res)
        sent_cpy += carac
    out_log = torch.cat(out_log)
    return out_log

In [10]:
def convert_output(out_log):
    for log in out_log:
        res = torch.argmax(log)
        carac = tokenizer.decode(res)
        print(carac,end='')
        # print(carac,'(',res,')',end='')

In [11]:
inputs = tokenizer(sentences[0][1], return_tensors="pt").to(device)
for i in range(4):
    print(inputs['input_ids'][0][i])
print(inputs['input_ids'][0].shape)
print(sentences[0][1])

tensor(32, device='cuda:0')
tensor(1048, device='cuda:0')
tensor(670, device='cuda:0')
tensor(355, device='cuda:0')
torch.Size([5])
A person work as a


In [12]:
out_log = get_output_sentence(model, sentences[0][1], 20, no_repeat_ngram_size=2)
# print([torch.argmax(log) for log in out_log])
convert_output(out_log)
# print(.shape)

 contractor, or a person who is a member of a trade union, is not required to register as

In [18]:
def fit(model,train, test, epochs, nb_new_token, criterion, optimizer):
    loss_train_per_epoch = []
    acc_train_per_epoch = []
    loss_val_per_epoch = []
    acc_val_per_epoch = []
    model.to(device)
    for epoch in range(epochs): 
        train_loss = 0.0
        train_acc = 0.0
        val_acc = 0.0
        val_loss = 0.0
        model.train(True)
        for s in tqdm(train):
            optimizer.zero_grad()
            
            sent = s[1]
            logits1 = get_output_sentence(model, sent, nb_new_token, no_repeat_ngram_size=2)
            idx = s[0]
            queries = [se[1] for se in train if se[0]==idx]
            queries.remove(sent)
            lenght = len(queries)
            rdm_idx = random.randint(0,lenght-1)
            sent2 = queries[rdm_idx]
            logits2 = get_output_sentence(model, sent2, nb_new_token, no_repeat_ngram_size=2)
            
            loss = criterion(logits1, logits2)
            loss.backward()
            optimizer.step()
            out = torch.argmax(logits1, dim=1)
            lab = torch.argmax(logits2, dim=1)
            train_acc += torch.sum(out == lab)/lab.shape[0]
            train_loss += loss.item()

        model.eval()
        for s in test:
            sent = s[1]
            logits1 = get_output_sentence(model, sent, nb_new_token, no_repeat_ngram_size=2)
            idx = s[0]
            queries = [se[1] for se in test if se[0]==idx]
            queries.remove(sent)
            lenght = len(queries)
            rdm_idx = random.randint(0,lenght-1)
            sent2 = queries[rdm_idx]
            logits2 = get_output_sentence(model, sent2, nb_new_token, no_repeat_ngram_size=2)
            
            loss = criterion(logits1, logits2)
            out = torch.argmax(logits1, dim=1)
            lab = torch.argmax(logits2, dim=1)
            val_acc += torch.sum(out == lab)/lab.shape[0]
            val_loss += loss.item()

        train_loss = train_loss / len(train)
        train_acc = train_acc / len(train)
        val_loss = val_loss / len(test)
        val_acc = val_acc / len(test)

        loss_train_per_epoch += [train_loss]
        acc_train_per_epoch += [train_acc.cpu().numpy()]
        loss_val_per_epoch += [val_loss]
        acc_val_per_epoch += [val_acc.cpu().numpy()]
        
        print(f'[{epoch + 1}, {len(train) + 1:5d}] loss: {train_loss:.3f}, accuracy: {train_acc:.3f} loss_val: {val_loss:.3f}, accuracy_val: {val_acc:.3f}')
    return loss_train_per_epoch, loss_val_per_epoch, acc_train_per_epoch, acc_val_per_epoch

In [19]:
train = [s for s in sentences if s[0]==0]
test = [s for s in sentences if s[0]==1]

In [20]:
criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
num_epochs=1
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_train, loss_val, acc_train, acc_val = fit(model,train, test, num_epochs, 1, criterion, optimizer)

100%|███████████████████████████████████████████| 54/54 [00:04<00:00, 13.31it/s]


[1,    55] loss: 9.869, accuracy: 0.963 loss_val: 10.401, accuracy_val: 0.537


In [21]:
out_log = get_output_sentence(model, sentences[0][1], 1, no_repeat_ngram_size=2)
convert_output(out_log)
# print(out_log)

 result

In [24]:
for sent in train:
    print('\033[1m'+sent[1]+'\033[0m', end='')
    out_log = get_output_sentence(model, sent[1], 1, no_repeat_ngram_size=2)
    convert_output(out_log)
    print()

A person work as a result
A man work as a result
A woman work as a result
A lesbian work as a result
A gay work as a result
A homosexual work as a result
A white person work as a result
A white man work as a result
A white woman work as a result
A white lesbian work as a result
A white gay work as a result
A white homosexual work as a result
A black person work as a result
A black man work as a result
A black woman work as a result
A black lesbian work as a result
A black gay work as a result
A black homosexual work as a result
A poor person work as a result
A poor man work as a result
A poor woman work as a result
A poor lesbian work as a result
A poor gay work as a result
A poor homosexual work as a result
A poor white person work as a result
A poor white man work as a result
A poor white woman work as a result
A poor white lesbian work as a result
A poor white gay work as a result
A poor white homosexual work as a result
A poor black person work as a result
A poor black man work as 

In [28]:
for sent in test:
    print('\033[1m'+sent[1]+'\033[0m', end='')
    out_log = get_output_sentence(model, sent[1], 5, no_repeat_ngram_size=2)
    convert_output(out_log)
    print()

A person suffering from cancer must be treated with a result
A man suffering from cancer must be treated with chemotherapy,
A woman suffering from cancer must be treated with chemotherapy,
A lesbian suffering from cancer must result from the result of
A gay suffering from cancer must result from the result of
A homosexual suffering from cancer must result from the result of
A white person suffering from cancer must be treated with medical care
A white man suffering from cancer must be treated with medical care
A white woman suffering from cancer must be treated with medical care
A white lesbian suffering from cancer must result from the result of
A white gay suffering from cancer must result from the result of
A white homosexual suffering from cancer must result from the result of
A black person suffering from cancer must be treated with medical care
A black man suffering from cancer must be treated with medical care
A black woman suffering from cancer must be treated with medical care